In [203]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import rayleigh, gilbrat, norm
import sys
import csv
import random
import time
from datetime import timedelta, date
from sqlalchemy import create_engine
import sqlite3
engine = create_engine('sqlite://', echo=False)


In [204]:
products_instances = pd.read_csv('C:/Users/ivan-/OneDrive/Desktop/materials/charge/business_product_instances.csv')
products=pd.read_csv('C:/Users/ivan-/OneDrive/Desktop/materials/charge/products.csv')
customers=pd.read_csv('../Customer/Customers.csv')
costed_events=pd.read_csv('C:/Users/ivan-/OneDrive/Desktop/materials/product/costed_events.csv')

In [205]:
customers.to_sql('Customers_table', con=engine)
products.to_sql('Products_table', con=engine)
products_instances.to_sql('Products_instances_table', con=engine)
costed_events.to_sql('Costed_events_table', con=engine)

In [206]:
#prepaid=engine.execute("""
#Select c.business_product_instance_id,1 as charge_couter,pi.activation_date,p.cumulative_charge,
#CASE
#    WHEN p.recurrent = False 
#        THEN 'Once'
#    WHEN p.recurrent = True
#        THEN 'Reccurent'
#    END AS recurrent 
#from Costed_events_table c
#join Products_instances_table pi on
#c.business_product_instance_id=pi.[Unnamed: 0]
#join Products_table p on
#pi.product_id=p.id
#where p.category='prepaid' and  c.date between pi.activation_date and pi.termination_date
#
#group by c.business_product_instance_id
#
#  """
#                        ).fetchall()
#
#

   

In [207]:
#postpaid=engine.execute("""
#Select c.business_product_instance_id, count(c.event_id) as charge_couter,pi.termination_date,sum(c.cost)+p.cumulative_charge,
#CASE
#    WHEN p.recurrent = False 
#        THEN 'Once'
#    WHEN p.recurrent = True
#        THEN 'Reccurent'
#    END AS recurrent 
#from Costed_events_table c
#join Products_instances_table pi on
#c.business_product_instance_id=pi.[Unnamed: 0]
#join Products_table p on
#pi.product_id=p.id
#where p.category='prepaid' and  c.date between pi.activation_date and pi.termination_date
#
#group by c.business_product_instance_id
#
#  """
#                         ).fetchall()


In [208]:
engine.execute("""
Create table Charge_table(
charge_id integer primary key autoincrement,
business_product_instance_id int,
charge_counter int,
date date,
cost float,
event_type varchar(20))

  """
              )

In [209]:
engine.execute("""
With prepaid as (Select c.business_product_instance_id,1 as charge_couter,pi.activation_date as date,p.cumulative_charge as cost,
CASE
    WHEN p.recurrent = False 
        THEN 'Once'
    WHEN p.recurrent = True
        THEN 'Reccurent'
    END AS event_type 
from Costed_events_table c
join Products_instances_table pi on
c.business_product_instance_id=pi.[Unnamed: 0]
join Products_table p on
pi.product_id=p.id
where p.category='prepaid' and  c.date between pi.activation_date and pi.termination_date

group by c.business_product_instance_id),


postpaid as(Select c.business_product_instance_id, count(c.event_id) as charge_couter,pi.termination_date as date,
sum(c.cost)+p.cumulative_charge as cost,
CASE
    WHEN p.recurrent = False 
        THEN 'Once'
    WHEN p.recurrent = True
        THEN 'Reccurent'
    END AS event_type 
from Costed_events_table c
join Products_instances_table pi on
c.business_product_instance_id=pi.[Unnamed: 0]
join Products_table p on
pi.product_id=p.id
where p.category='postpaid' and  c.date between pi.activation_date and pi.termination_date

group by c.business_product_instance_id),

all_charges as (
Select *from prepaid
union all
Select *from postpaid)

INSERT INTO Charge_table (business_product_instance_id ,
charge_counter ,
date ,
cost ,
event_type )  SELECT *from all_charges;


  """
                         )

In [210]:
charges_all_finally=engine.execute("""
select *from Charge_table
  """    ).fetchall()

In [211]:
engine.execute("""
drop table Charge_table
  """
                         )

In [212]:
charges_df=pd.DataFrame(charges_all_finally,
                    columns=['charge_id',
'business_product_instance_id',
'charge_counter',
'date',
'cost',
'event_type',
 ])
charges_df

,charge_id,business_product_instance_id,charge_counter,date,cost,event_type
0,1,24,1,2022-02-23,148.5,Once
1,2,25,1,2022-02-11,148.5,Once


In [213]:
charges_df.to_csv('charges.csv')